In [143]:
!pip3 install contractions
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
import re
import contractions
import nltk

nltk.download('words') # download list of english words
nltk.download('stopwords') # download list of stopwords
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('omw-1.4')

from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from nltk.corpus import stopwords

stopWords = stopwords.words('english')
englishWords = set(nltk.corpus.words.words())

[nltk_data] Downloading package words to /Users/amritapal/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/amritapal/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/amritapal/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/amritapal/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/amritapal/nltk_data...


In [144]:
# using this link to create this project -> figuring out what libraries to use!
# https://www.simplilearn.com/tutorials/machine-learning-tutorial/how-to-create-a-fake-news-detection-system

In [145]:
true_df = pd.read_csv("data/True.csv")
# true_df.head(10)

In [146]:
fake_df = pd.read_csv("data/Fake.csv")
# fake_df.head(10)

In [147]:
# parameters for true/false
true_df['truthVal'] = 1
fake_df['truthVal'] = 0

In [148]:
# should manually test data at some point

In [149]:
merged_df = pd.concat([true_df, fake_df], axis = 0)

In [150]:
# dropping unnecessary columns

merged_df = merged_df.drop(['title', 'subject', 'date'], axis = 1)
#merged_df

In [151]:
# pre-processing link to clean text: https://jon-dagdagan.medium.com/fake-news-detection-pre-processing-text-d9648a2854e5

def lowercase_and_remove_URL(text):
    text = text.lower()
    text = re.sub(r'https?://\S+', '', text)
    return text


def remove_contractions(text):
    return ' '.join([contractions.fix(word) for word in text.split()])


tokenizer = RegexpTokenizer(r'\w+')

def tokenize_text(text):
    tokens = tokenizer.tokenize(text)
    return tokens


# chose to lemmatize instead of stem
lemmatizer = WordNetLemmatizer()

# Convert the nltk pos tags to tags (word classes) that wordnet can recognize
def nltkToWordnet(nltk_tag):
  if nltk_tag.startswith('J'):
    return wordnet.ADJ
  elif nltk_tag.startswith('V'):
    return wordnet.VERB
  elif nltk_tag.startswith('N'):
    return wordnet.NOUN
  elif nltk_tag.startswith('R'):
    return wordnet.ADV
  else:
    return None # words that don't need to be reduced to their root word

# Lemmatize a list of words/tokens
def lemmatize(tokens):
  pos_tags = nltk.pos_tag(tokens) # nltk version of word classes
  reduced_words = []
  for word, tag in pos_tags:
    tag = nltkToWordnet(tag)
    if tag is None:
      reduced_words.append(word)
    else:
      reduced_words.append(lemmatizer.lemmatize(word, tag))
  return reduced_words


def remove_stopWords(tokens):
    return [word for word in tokens if (word in englishWords and word not in stopWords)]


def clean_text(text):
    text = lowercase_and_remove_URL(text)
    text = remove_contractions(text)
    tokens = tokenize_text(text)
    reduced_words = lemmatize(tokens)
    reduced_words = remove_stopWords(reduced_words)
    return ' '.join(reduced_words)

In [152]:
# cleaning merged_df text
merged_df['text'] = merged_df['text'].apply(clean_text)
merged_df.head(10)

,text,truthVal
0,head conservative republican faction congress ...,1
1,people allow first time enlist military starti...,1
2,special counsel investigation link russia pres...,1
3,trump campaign adviser tell diplomat may russi...,1
4,president trump call postal service charge muc...,1
5,west palm beach white house say set kick talk ...,1
6,west palm beach president trump say believe fa...,1
7,following statement post twitter account presi...,1
8,following statement post twitter account presi...,1
9,secretary state say certify democratic senator...,1


In [153]:
# the code above took an extremely long time to run! (note for the future as I improve code)